# Create control file
Populates a text file with the required inputs for a mizuRoute run.

In [1]:
# modules
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Control file handling

In [2]:
# Easy access to control file folder
controlFolder = Path('../../../0_controlFiles')

In [3]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [4]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    with open(file) as contents:
        for line in contents:
            
            # ... find the line with the requested setting
            if setting in line:
                break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
       
    # Return this value    
    return substring

In [5]:
# Function to specify a default path
def make_default_path(suffix):
    
    # Get the root path
    rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )
    
    # Get the domain folder
    domainName = read_from_control(controlFolder/controlFile,'domain_name')
    domainFolder = 'domain_' + domainName
    
    # Specify the forcing path
    defaultPath = rootPath / domainFolder / suffix
    
    return defaultPath

#### Find where the control file needs to go

In [6]:
# Forcing file list path & name
control_path = read_from_control(controlFolder/controlFile,'settings_mizu_path')
control_name = read_from_control(controlFolder/controlFile,'settings_mizu_control_file')

In [7]:
# Specify default path if needed
if control_path == 'default':
    control_path = make_default_path('settings/mizuRoute') # outputs a Path()
else:
    control_path = Path(control_path) # make sure a user-specified path is a Path()

In [8]:
# Make the folder if it doesn't exist
control_path.mkdir(parents=True, exist_ok=True)

#### Read the required information from control_active.txt

In [15]:
# Get the experiment ID
experiment_id = read_from_control(controlFolder/controlFile,'experiment_id')

In [11]:
# Paths - settings folder
path_to_settings = read_from_control(controlFolder/controlFile,'settings_mizu_path')

# Specify default path if needed
if path_to_settings == 'default':
    path_to_settings = make_default_path('settings/mizuRoute') # outputs a Path()
else:
    path_to_settings = Path(path_to_settings) # make sure a user-specified path is a Path()

In [16]:
# Paths - SUMMA output/mizuRoute input folder
path_to_input = read_from_control(controlFolder/controlFile,'experiment_output_summa')

# Specify default path if needed
if path_to_input == 'default':  
    path_to_input = make_default_path('simulations/' + experiment_id + '/SUMMA') # outputs a Path()
else:
    path_to_input = Path(path_to_input) # make sure a user-specified path is a Path()

In [18]:
# Paths - mizuRoute output folder
path_to_output = read_from_control(controlFolder/controlFile,'experiment_output_mizuRoute')

# Specify default path if needed
if path_to_output == 'default':  
    path_to_output = make_default_path('simulations/' + experiment_id + '/mizuRoute') # outputs a Path()
else:
    path_to_output = Path(path_to_output) # make sure a user-specified path is a Path()
    
# Make the folder if it doesn't exist
path_to_output.mkdir(parents=True, exist_ok=True)

In [24]:
# Simulation times
sim_start = read_from_control(controlFolder/controlFile,'experiment_time_start')
sim_end   = read_from_control(controlFolder/controlFile,'experiment_time_end')

# Define default times if needed
if sim_start == 'default':
    raw_time = read_from_control(controlFolder/controlFile,'forcing_raw_time') # downloaded forcing (years)
    year_start,_ = raw_time.split(',') # split into separate variables
    sim_start = year_start + '-01-01 00:00' # construct the filemanager field

if sim_end == 'default':
    raw_time = read_from_control(controlFolder/controlFile,'forcing_raw_time') # downloaded forcing (years)
    _,year_end = raw_time.split(',') # split into separate variables
    sim_end   = year_end   + '-12-31 23:00' # construct the filemanager field

In [34]:
# Topology settings
topology_nc  = read_from_control(controlFolder/controlFile,'settings_mizu_topology')

# Variables below are hard-coded in 1_create_network_topology.py to be consistent with mizuRoute docs
topology_seg = 'seg' 
topology_hru = 'hru' 
topology_outlet = -9999

In [78]:
# Remap settings
remap_flag = read_from_control(controlFolder/controlFile,'river_basin_needs_remap')
if remap_flag.lower() == 'yes':
    do_remap          = 'T'
    remap_nc          = read_from_control(controlFolder/controlFile,'settings_mizu_remap')
    
    # Variables below are hard-coded in 1_remap_summa_catchments_to_routing.py to be consistent with mizuRoute docs
    remap_var_rn_hru  = 'RN_hruId' 
    remap_var_weight  = 'weight' 
    remap_var_hm_gru  = 'HM_hruId'
    remap_var_overlap = 'nOverlaps'
    remap_dim_hm_gru  = 'hru'
    remap_dim_data    = 'data'
else:
    do_remap = 'F'

In [89]:
# SUMMA output settings
routing_nc  = experiment_id + '_timestep.nc'
routing_var_flow = read_from_control(controlFolder/controlFile,'settings_mizu_routing_var')
routing_var_flow_units = read_from_control(controlFolder/controlFile,'settings_mizu_routing_units')
routing_dt = read_from_control(controlFolder/controlFile, 'settings_mizu_routing_dt')

# Variables below are hard-coded in SUMMA
routing_dim_time = 'time'  
routing_var_time = 'time'  
routing_dim_id  = 'gru' 
routing_var_id  = 'gruId'

In [59]:
# Misc settings
output_freq = read_from_control(controlFolder/controlFile,'settings_mizu_output_freq')
do_basin_route = read_from_control(controlFolder/controlFile,'settings_mizu_within_basin')

#### Make the file

In [90]:
# Create the file list
with open(control_path / control_name, 'w') as cf:
    
    # Header
    cf.write("! mizuRoute control file generated by SUMMA public workflow scripts \n")
    
    # Folders
    cf.write("\n! --- DEFINE DIRECTORIES \n")
    cf.write("<ancil_dir>             {}/    ! Folder that contains ancillary data (river network, remapping netCDF) \n".format(path_to_settings))
    cf.write("<input_dir>             {}/    ! Folder that contains runoff data from SUMMA \n".format(path_to_input))
    cf.write("<output_dir>            {}/    ! Folder that will contain mizuRoute simulations \n".format(path_to_output))
    
    # Base parameters
    cf.write("\n! --- NAMELIST FILENAME \n")
    cf.write("<param_nml>             param.nml.default    ! Spatially constant parameter namelist (should be stored in <ancil_dir>) \n")
    
    # Simulation settings
    cf.write("\n! --- DEFINE SIMULATION CONTROLS \n")
    cf.write("<case_name>             {}    ! Simulation case name. This used for output netCDF, and restart netCDF name \n".format(experiment_id))
    cf.write("<sim_start>             {}    ! Time of simulation start. format: yyyy-mm-dd or yyyy-mm-dd hh:mm:ss \n".format(sim_start))
    cf.write("<sim_end>               {}    ! Time of simulation end. format: yyyy-mm-dd or yyyy-mm-dd hh:mm:ss \n".format(sim_end))
    cf.write("<newFileFrequency>      {}    ! Frequency for new output files (single, day, month, or annual) \n".format(output_freq))
    
    # Topology file
    cf.write("\n! --- DEFINE TOPOLOGY FILE AND DIMENSIONS \n")
    cf.write("<fname_ntopOld>         {}    ! Name of input netCDF for River Network \n".format(topology_nc))
    cf.write("<dname_sseg>            {}    ! Dimension name for reach in river network netCDF \n".format(topology_seg))
    cf.write("<dname_nhru>            {}    ! Dimension name for RN_HRU in river network netCDF \n".format(topology_hru))
    cf.write("<seg_outlet>            {}    ! Downstream segment ID that indicates the outlet \n".format(topology_outlet))
    
    # SUMMA output
    cf.write("\n! --- DEFINE RUNOFF FILE \n")
    cf.write("<fname_qsim>            {}    ! netCDF name for HM_HRU runoff \n".format(routing_nc))
    cf.write("<vname_qsim>            {}    ! Variable name for HM_HRU runoff \n".format(routing_var_flow))
    cf.write("<units_qsim>            {}    ! Units of input runoff. e.g., mm/s \n".format(routing_var_flow_units)) 
    cf.write("<dt_qsim>               {}    ! Time interval of input runoff in seconds, e.g., 86400 sec for daily step \n".format(routing_dt)) 
    cf.write("<dname_time>            {}    ! Dimension name for time \n".format(routing_dim_time))
    cf.write("<vname_time>            {}    ! Variable name for time \n".format(routing_var_time))
    cf.write("<dname_hruid>           {}    ! Dimension name for HM_HRU ID \n".format(routing_dim_id)) 
    cf.write("<vname_hruid>           {}    ! Variable name for HM_HRU ID \n".format(routing_var_id))
    
    # Remapping
    cf.write("\n! --- DEFINE RUNOFF MAPPING FILE \n")
    cf.write("<is_remap>              {}    ! Logical to indicate runoff needs to be remapped to RN_HRU. T or F \n".format(do_remap))
    
    if remap_flag.lower() == 'yes':
        cf.write("<fname_remap>           {}    ! netCDF name of runoff remapping \n".format(remap_nc))
        cf.write("<vname_hruid_in_remap>  {}    ! Variable name for RN_HRUs \n".format(remap_var_rn_hru))
        cf.write("<vname_weight>          {}    ! Variable name for areal weights of overlapping HM_HRUs \n".format(remap_var_weight))
        cf.write("<vname_qhruid>          {}    ! Variable name for HM_HRU ID \n".format(remap_var_hm_gru))
        cf.write("<vname_num_qhru>        {}    ! Variable name for a numbers of overlapping HM_HRUs with RN_HRUs \n".format(remap_var_overlap))
        cf.write("<dname_hru_remap>       {}    ! Dimension name for HM_HRU \n".format(remap_dim_hm_gru))
        cf.write("<dname_data_remap>      {}    ! Dimension name for data \n".format(remap_dim_data))
    
    # Misc settings
    cf.write("\n! --- MISCELLANEOUS \n")
    cf.write("<doesBasinRoute>   {}    ! Hillslope routing options. 0 -> no (already routed by SUMMA), 1 -> use IRF \n".format(do_basin_route)) # only for routing option 2

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [82]:
# Set the log path and file name
logPath = control_path
log_suffix = '_make_control_file.txt'

In [83]:
# Create a log folder
logFolder = '_workflow_log'
Path( logPath / logFolder ).mkdir(parents=True, exist_ok=True)

In [84]:
# Copy this script
thisFile = '1_create_control_file.ipynb'
copyfile(thisFile, logPath / logFolder / thisFile);

In [85]:
# Get current date and time
now = datetime.now()

In [86]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + log_suffix
with open( logPath / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated control file.']
    for txt in lines:
        file.write(txt) 